In [2]:
import torch

x = torch.arange(4.0, requires_grad=True) # 声明requires_grad后可以通过grad属性查看梯度值
x

tensor([0., 1., 2., 3.], requires_grad=True)

创建的tensor是计算图中的叶子节点

In [3]:
x.grad,x.grad_fn # 查看梯度，y关于x的导数放在这里

(None, None)

### 简单例子：求y=2XTX的梯度，梯度应为4X
手动演算的方法：画出计算图，进行符号求导（设x=[x1,x2,x3,x4] y是标量，标量对向量求导是向量，求出导函数后带入x的值，便可得所要的梯度）

In [4]:
y = 2 * torch.dot(x, x) # y是关于x的标量函数，未知数是x，输出是个标量
y   # y.grad_fn非空代表这个tensor是计算出来的

tensor(28., grad_fn=<MulBackward0>)

In [5]:
# 调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward()
x.grad  # 查看梯度

tensor([ 0.,  4.,  8., 12.])

In [6]:
x.grad == 4 * x

tensor([True, True, True, True])

In [7]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_() # 方法后以'_'结尾代表原地修改值
y = x.sum()
# 调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 非标量变量的反向传播（向量对向量求导=矩阵）
在pytorch中有个简单的规定，不让向量对向量求导，只允许标量对向量求导。因此，一个向量调用backward()，则需要传入一个gradient参数。传入这个参数就是为了把向量对向量的求导转换为标量对向量的求导。

这里，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。

In [8]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x))) 
# 由于 y @ torch.ones(len(x)) 就是 y.sum() 为标量，随后再backward()
y.sum().backward()  
x.grad

tensor([0., 2., 4., 6.])

### 分离计算

In [17]:
x.grad.zero_()
y = x * x
u = y.detach() # 把y这个tensor的值给u，u不进入计算图，u作为常数处理,u的值与y相同
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

## 控制流的梯度计算

In [9]:
def f(a):
    b = a * 2
    while b.norm() < 1000: # norm()是范数：L1范数：所有元素的绝对值的求和 L2范数：所有元素的绝对值的平方和的开方
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [30]:
a = torch.randn(size=(), requires_grad=True)
a

tensor(1.6182, requires_grad=True)

In [31]:
d = f(a)
d.backward()

In [33]:
a.grad == d / a , a.grad

(tensor(True), tensor(1024.))

### 小结

深度学习框架可以自动计算导数：我们首先将梯度附加到想要对其计算偏导数的变量上 require_grad = True，然后框架会跟踪这个tensor的计算，为我们保存相关的计算图，执行某个ta的反向传播函数，并访问得到的梯度。